# Quick view of x1-x2 slit scan

This notebook is currently being used for scouting scans.

In [ ]:
import sys
import os
from os.path import join
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from scipy import ndimage
import proplot as pplt

from tools import image_processing as ip
from tools import utils

sys.path.append('/Users/46h/Research/btf/btf-scripts/')
import scan_patterns as sp

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['cmap.discrete'] = False

In [ ]:
datadir = './Diagnostics/Data/Measurements/2022-04-29/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220429125723-x2d-slit.h5'
file = h5py.File(join(datadir, filename), 'r')
data = file['scandata']
pprint(data.dtype.fields)

In [ ]:
acts = ['xp_PositionSync', 'x_PositionSync']
signame = 'cam34_integral'
sdiag = ['cam34_integral', 'cam34_saturation', 'bcm04']
signal = data[signame]

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
cam = 'cam34'
cam_settings = ip.camera_settings(cam)
nx = cam_settings.nx
ny = cam_settings.ny
print(f'ny, nx = ({ny}, {nx})')

In [ ]:
fig, ax = pplt.subplots(figsize=(8, 2))
for act in acts:
    ax.plot(data[:, act], marker=None, label=act)  # plt.plot(tpred,act3,':',color='C2')
ax.legend()
ax.format(xlabel='Step', ylabel='[mm]')

In [ ]:
fig, ax = pplt.subplots()
ax.scatter(data[:, acts[1]], data[:, acts[0]], marker='s', s=50,
           c=np.log10(1.0 + 10.0 * signal),  
           colorbar=True, colorbar_kw=dict(label=f'log10({signame})'))
ax.format(xlabel=acts[1], ylabel=acts[0])
plt.savefig('_output/x2d')

In [ ]:
for item in sdiag:
    print(f'Diagnostic: {item}')
    print('Max = {:.3f}'.format(np.max(data[:, item])))
    print('Min = {:.3f}'.format(np.min(data[:, item])))
    print('Mean = {:.3f}'.format(np.mean(data[:, item])))
    fig, ax = pplt.subplots(figsize=(7.0, 2.0))
    ax.plot(data[:, item], color='black', marker='.', ms=0, label=item)
    ax.format(xlabel='Step', ylabel=item)
    plt.show()

#### Pick frames for background calculation

In [ ]:
bgidx = np.arange(1, 35)

fig, ax = pplt.subplots()
ax.plot(data[:, acts[1]], data[:, acts[0]], alpha=0.2)
ax.scatter(data[:, acts[1]], data[:, acts[0]], marker='s', 
           c=np.log10(1.0 + 10.0 * signal),
           colorbar=True, colorbar_kw=dict(label=f'log10({signame})'))

ax.plot(data[bgidx, acts[1]], data[bgidx, acts[0]], 'rs', label='Background')
ax.legend()
ax.format(xlabel=acts[1], ylabel=acts[0])

## Threshold and background


In [ ]:
bg_lvl_est = np.max(data[bgidx, signame])
print(f'Background level from integral: {bg_lvl_est:.6f}')

thr_lvl_est = bg_lvl_est / np.max(data[:, signame])
print(f'Est. D.R. 10^{np.log10(thr_lvl_est):.3f}')

bg_mean = data[0, f'{cam}_image']
for i in bgidx[1:]:
    bg_mean += data[i, f'{cam}_image']
    
nbg = len(bgidx)
bg_mean = bg_mean / nbg
bg_ima = np.reshape(bg_mean,[nx,ny])

fig, ax = pplt.subplots()
ax.pcolormesh(bg_ima, colorbar=True)
ax.format(title='averaged background images')

In [ ]:
# thr = 0.043
thr = 0.11
w = signal.copy()
w[w < thr] = 0.0
print(f'signame = {signame}')
print('Est 4D dynamic range 10^%.3f'%np.log10(thr / max(w)))

In [ ]:
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.semilogy(data[:, signame], color='black', marker='.', ms=4)
ax.semilogy(w, marker='.', s=4, color='red', lw=0, label='Above thresh')
ax.legend()
ax.format(xlabel='Step', ylabel=signame)

In [ ]:
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.semilogy(np.sort(data[:, signame]), color='black')
ax.semilogy(np.sort(w), marker='.', color='red', lw=0, label='Above thresh')
ax.legend()
ax.format(xlabel='Step', ylabel=signame,)

### Plot measurement with threshold

In [ ]:
fig, ax = pplt.subplots()
ax.scatter(data[:, acts[1]], data[:, acts[0]], marker='s', s=50,
           c=np.log10(1.0 + 10.0 * w),
           colorbar=True)
ax.format(xlabel=acts[1], ylabel=acts[0])

### Frame with peak signal

In [ ]:
idx = np.argmax(w)
max_pixel = np.max(data[idx, f'{cam}_image'])

print('peak: ')
for item in list(acts) + list(sdiag):
    print(f'  {item} = {data[idx, item]:.3f} [mm]')
print(f'Max pixel: {max_pixel}')

### Estimate dynamic range by examining frame with peak signal

In [ ]:
idx = np.argmax(w)
im = data[idx, cam + '_image'].reshape(ny, nx)
pix_max = np.max(im)
print(f'Max pixel value {pix_max}')

In [ ]:
norm = 'log'  # {'log', None}
fig, axes = pplt.subplots(ncols=2, sharey=False)
axes[0].pcolormesh(im, norm=norm, colorbar=True)
axes[1].plot(np.sum(im, axis=0), color='black')
axes[1].format(yscale='log')
plt.show()

In [ ]:
# mask = np.ones(np.shape(im))
# mask[400:, :] = 0.0
# mask[:100, :] = 0.0
# mask[:, 480:] = 0.0
# mask[:, :200] = 0.0
# immask = im * mask

# print('5D DR: 10^%.3f'%(np.log10(np.max(mask * bg_ima) / np.max(immask))))
# print('4D DR: 10^%.3f'%(np.log10(np.sum(mask * bg_ima) / np.sum(immask))))

In [ ]:
# fig, axes = pplt.subplots(ncols=2, sharey=False)
# axes[0].pcolormesh(np.log10(immask))
# axes[0].annotate('(logscale)', xy=(1, 1), color='white')
# axes[1].plot(np.sum(im, axis=0))
# plt.show()

In [ ]:
# thr2 = 1e5

# plt.figure()
# imathr = im.copy()
# imathr[imathr < thr2] = 0.0
# plt.pcolor(np.log10(imathr))

# print('naive 5D DR: 10^%.3f'%(np.log10(thr2/imathr.max())))
# print('naive 4D DR: 10^%.3f'%(np.log10(thr2/imathr.sum())))
# print('5D DR: 10^%.3f'%(np.log10(bg_ima.max()/im.max())))
# print('4D DR: 10^%.3f'%(np.log10(bg_ima.sum()/im.sum())))

#### Threshold after smoothing

In [ ]:
# ima = im
# ima_smooth = ndimage.gaussian_filter(ima, 5, mode='nearest')

# thr2 = 50000.0
# plt.figure()
# imathr = ima_smooth.copy()
# imathr[imathr < thr2] = 0.0
# plt.pcolor(np.log10(imathr))

# print('naive 5D DR: 10^%.3f'%(np.log10(thr2 / imathr.max())))
# print('naive 4D DR: 10^%.3f'%(np.log10(thr2 / imathr.sum())))
# print('5D DR: 10^%.3f'%(np.log10(bg_ima.max() / ima.max())))
# print('4D DR: 10^%.3f'%(np.log10(bg_ima.sum() / ima.sum())))

## Measure against new scan boundaries

Make a list of coordinates that have signal.

In [ ]:
pts = np.vstack([data[:, 'xp_PositionSync'], data[:, 'x_PositionSync']]).T
sig_idx = w > 0.0
sig_pts = pts[sig_idx, :]

Define "normalized" actuator coordinates.

In [ ]:
ndim = 2
M = np.identity(ndim)
M[0, 1] = 1.9

# xp
d1_center = 15.0
d1_distance = 24.0
d1_steps = 32

# x
d2_center = 21.0
d2_distance = 17.5
d2_steps = 32

reprate = 5
navg = 1

In [ ]:
def gen(start=0): 
    center = np.array([d1_center, d2_center])
    distance = np.array([d1_distance, d2_distance])
    nsteps = np.array([d1_steps, d2_steps])
    stepsize = distance / nsteps
    counter = 0
        
    ##### STEPPER SCAN COMMANDS
    if navg > 0:             
        for point in sp.grid(center,distance,nsteps): 
            realpoint = np.matmul(M,point-center) + center

            counter += 1
            if counter > start:
                yield realpoint, navg
        
    ##### SWEEPER SCAN COMMANDS
    elif navg == 0:
        sweep_speed = distance[0]/nsteps[0]*reprate
        nsteps[0] = 1

        lastpoint = next(sp.grid(center,distance,nsteps))
        for point in sp.grid(center,distance,nsteps): 
            # -- skip step if sweeper is not moving
            if (point[0] - lastpoint[0]) == 0:
                lastpoint = point.copy()
                continue

            # -- send command if sweeper is moving on this step
            else: 
                # -- first transform to real coordinates
                realpoint = np.matmul(M,point-center) + center
                reallastpoint = np.matmul(M,lastpoint-center) + center

                # -- build up cmd list
                cmd = []
                # -- sweeper command
                cmd.append((reallastpoint[0],realpoint[0],sweep_speed))
                # -- commands for the rest of the actuators
                for j in range(1,ndim):
                    # if slope, stepper may have to sweep as well
                    stepper_speed = sweep_speed * np.abs(realpoint[j] - reallastpoint[j]) / distance[0]
                    cmd.append((reallastpoint[j],realpoint[j],stepper_speed))
            
                lastpoint = point.copy()
            counter += 1
            if counter > start:        
                yield cmd

In [ ]:
# Run points generator
lgen = list(gen())
new_pts = np.zeros([len(lgen), ndim])
for i in range(len(lgen)):
    new_pts[i, :] = lgen[i][0]
    
# Un-shear new (generated) points.
center = np.array([d1_center, d2_center])
Minv = np.linalg.inv(M)
new_pts_n = utils.apply(Minv, new_pts - center) + center
new_pts_n = utils.apply(Minv, new_pts - center) + center

# Un-shear signal points.
sig_pts_n = np.zeros(sig_pts.shape)
for row in range(sig_pts.shape[0]):
    sig_pts_n[row, :] = np.matmul(Minv, sig_pts[row, :] - center) + center

In [ ]:
coords = np.vstack([data[:, acts[0]], data[:, acts[1]]]).T
coords_n = np.zeros(coords.shape)
for row in range(coords.shape[0]):
    coords_n[row, :] = np.matmul(Minv, coords[row, :] - center) + center

In [ ]:
fig, axes = pplt.subplots(ncols=2, sharex=False, sharey=False,
                          wspace=8.0, figwidth=8.0)
for ax, _pts, _new_pts in zip(axes, [sig_pts, sig_pts_n], [new_pts, new_pts_n]):
    ax.scatter(_pts[:, 0], _pts[:, 1], marker='s', color='black', label='signal')
    ax.plot(_new_pts[:, 0], _new_pts[:, 1], marker='.', s=4, color='grey', label='scan')
axes.format(xlabel=acts[0], ylabel=acts[1], toplabels=['Sheared', 'Un-sheared'])
axes[0].axvline(49.0, color='red', label='limit')
axes[0].legend(ncols=1)
plt.show()